### upload information into bq

In [1]:
import pandas as pd
import numpy as np
import json
from google.cloud import bigquery
from google.cloud import storage
from pathlib import Path
pd.set_option('display.max_columns', 500)

#### folders

In [43]:
input_path = Path.cwd() / '../data' / 'input'
output_path = Path.cwd() / '../data' / 'output'

In [3]:
input_path

PosixPath('/home/jupyter/projects/cable-theft/notebooks/../data/input')

#### read cable_theft.csv

In [4]:
csv_filename = input_path / "cable_theft.csv"

In [5]:
csv_filename

PosixPath('/home/jupyter/projects/cable-theft/notebooks/../data/input/cable_theft.csv')

In [6]:
cable_theft_df = pd.read_csv(csv_filename, parse_dates=['date 3'],infer_datetime_format=True)

In [7]:
cable_theft_df.columns

Index(['SIR Number', 'Year', 'Month', 'Day', 'Hour', 'DoW', 'Reported Date',
       'FY', 'Week ', 'Exchange', 'Exch hits', 'CSS DB', 'NED1',
       'Damage Est Nu', 'Postcode', 'Address', 'Information about Incident',
       'Location Details', 'Cut/ Theft', 'Cable  Size   TOTAL  pairs ONLY',
       'Live pairs', 'Total Length metres', 'Cable affected details',
       'E side, D side, Fibre', 'Police Force', 'Police Reference Number',
       'N#I#O's Name /\nOp Name', 'County', 'region', 'Cable size',
       'Indicative copper worth based on current UK copper price',
       'cost (raptor)',
       'Incident form rerturned or response received in X days', 'Hrs claimed',
       'Costs (excl MHours)', 'Manhour Cost', 'Total Cost',
       'Flagged to insurance', 'Hrs claimed on NJR', '#', 'Est Number', 'PIA',
       'NOTES ', 'date 3', 'date mmm yy', 'Repeat Hit',
       'Cable anchors in this Fin Year'],
      dtype='object')

In [8]:
cable_theft_df.head()

,SIR Number,Year,Month,Day,Hour,DoW,Reported Date,FY,Week,Exchange,Exch hits,CSS DB,NED1,Damage Est Nu,Postcode,Address,Information about Incident,Location Details,Cut/ Theft,Cable Size TOTAL pairs ONLY,Live pairs,Total Length metres,Cable affected details,"E side, D side, Fibre",Police Force,Police Reference Number,N#I#O's Name /\nOp Name,County,region,Cable size,Indicative copper worth based on current UK copper price,cost (raptor),Incident form rerturned or response received in X days,Hrs claimed,Costs (excl MHours),Manhour Cost,Total Cost,Flagged to insurance,Hrs claimed on NJR,#,Est Number,PIA,NOTES,date 3,date mmm yy,Repeat Hit,Cable anchors in this Fin Year
0,11009105,2001,11,9,0,Fri,37204.0,2001-2002,32,Adlington,2,LC,NN-NORTHWESTEAST,NaN,PR6 9NA,"Bolton Road, Adlington, Chorley",Report states that cables were cut to facilita...,NaN,Cut,20.0,20.0,0,NaN,d,Lancashire Constabulary,NaN,NaN,North West,Preston,< 100 pr,NaN,-,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,2001-09-11,2001 Wk: 32,1,0
1,11005559,2002,8,16,0,Fri,37484.0,2002-2003,20,Horsham,9,SD,NL-WESTDOWNS,NaN,RH12 3RR,"Robin Hood Lane, Warnham, Horsham",Snare alarm activated and Police attended. Evi...,NaN,Cut,1700.0,1700.0,0,NaN,e,Surrey Police,NaN,NaN,South East,Redhill,NaN,NaN,-,Not Sent,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,2002-08-16,2002 Wk: 20,1,0
2,11005565,2002,8,16,0,Fri,37484.0,2002-2003,20,Downton,4,ST,NX-AVONWILTS,NaN,SP5 3HN,"Breamore Road, Downton, Salisbury",NaN,NaN,Theft,600.0,600.0,0,NaN,e,Wiltshire Police,NaN,NaN,South West,Salisbury,NaN,NaN,-,Not Sent,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,2002-08-16,2002 Wk: 20,1,0
3,7000132,2007,4,3,0,Tue,39175.0,2007-2008,1,Wrexham,4,WN,NW-CHESTERMERSEYSIDE,NaN,LL14 6PY,"Ruabon Road, Ruabon",Call to report two males seen in a white Renau...,NaN,Intelligence,NaN,NaN,0,NaN,d,North Wales Police,NaN,NaN,Wales,Llandudno,NaN,NaN,-,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,2007-03-04,2007 Wk: 1,1,0
4,7000158,2007,4,4,0,Wed,39176.0,2007-2008,1,Collyhurst,142,MR,NN-NORTHWESTWEST,NaN,M9 5RP,"Oakridge Walk,",Operation Eiger: The caller is reporting damag...,NaN,Theft,NaN,NaN,0,NaN,e,Greater Manchester Police,NaN,NaN,North West,Manchester,NaN,NaN,-,Not Sent,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,2007-04-04,2007 Wk: 1,60,0


#### clean column names

In [10]:
cable_theft_df.rename(columns={"#": "Number"},inplace=True)

In [11]:
cable_theft_df.columns = cable_theft_df.columns.str.lower().str.strip().str.replace(r'\n','').str.replace("[/,',(,),#]", "").str.replace('  ', ' ').str.replace(' ', '_').str.replace('__', '_') 

/tmp/ipykernel_29111/1639713784.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  cable_theft_df.columns = cable_theft_df.columns.str.lower().str.strip().str.replace(r'\n','').str.replace("[/,',(,),#]", "").str.replace('  ', ' ').str.replace(' ', '_').str.replace('__', '_')
/tmp/ipykernel_29111/1639713784.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  cable_theft_df.columns = cable_theft_df.columns.str.lower().str.strip().str.replace(r'\n','').str.replace("[/,',(,),#]", "").str.replace('  ', ' ').str.replace(' ', '_').str.replace('__', '_')


In [12]:
cable_theft_df.columns

Index(['sir_number', 'year', 'month', 'day', 'hour', 'dow', 'reported_date',
       'fy', 'week', 'exchange', 'exch_hits', 'css_db', 'ned1',
       'damage_est_nu', 'postcode', 'address', 'information_about_incident',
       'location_details', 'cut_theft', 'cable_size_total_pairs_only',
       'live_pairs', 'total_length_metres', 'cable_affected_details',
       'e_side_d_side_fibre', 'police_force', 'police_reference_number',
       'nios_name_op_name', 'county', 'region', 'cable_size',
       'indicative_copper_worth_based_on_current_uk_copper_price',
       'cost_raptor', 'incident_form_rerturned_or_response_received_in_x_days',
       'hrs_claimed', 'costs_excl_mhours', 'manhour_cost', 'total_cost',
       'flagged_to_insurance', 'hrs_claimed_on_njr', 'number', 'est_number',
       'pia', 'notes', 'date_3', 'date_mmm_yy', 'repeat_hit',
       'cable_anchors_in_this_fin_year'],
      dtype='object')

#### sir_number

In [13]:
# leave a string - some end in 'a'

#### incident_dt

In [14]:
cable_theft_df['incident_dt'] = pd.to_datetime(dict(year=cable_theft_df.year, month=cable_theft_df.month, day=cable_theft_df.day))

#### day_of_week

In [15]:
cable_theft_df['day_of_week'] = cable_theft_df['incident_dt'].dt.dayofweek

In [16]:
# drop dow
cable_theft_df.drop("dow", axis=1,  inplace=True)

#### cut_theft

In [17]:
cable_theft_df['cut_theft'] = cable_theft_df['cut_theft'].str.lower().str.strip()

#### convert to float (as it preserves nulls)

In [18]:
cable_theft_df['cable_size_total_pairs_only'] = cable_theft_df['cable_size_total_pairs_only'].astype(float)

In [19]:
cable_theft_df['live_pairs'] = cable_theft_df['live_pairs'].astype(float)

In [20]:
cable_theft_df['total_length_metres'] = cable_theft_df['total_length_metres'].str.lower().str.replace('unknown','0').str.replace('y','0').astype(float)

In [21]:
cable_theft_df['indicative_copper_worth_based_on_current_uk_copper_price'] = cable_theft_df['indicative_copper_worth_based_on_current_uk_copper_price'].astype(float)

In [22]:
cable_theft_df['total_cost'] = cable_theft_df['total_cost'].astype(float)

In [23]:
cable_theft_df['costs_excl_mhours'] =cable_theft_df['costs_excl_mhours'].astype(float)

In [24]:
cable_theft_df['hrs_claimed_on_njr'] =cable_theft_df['hrs_claimed_on_njr'].astype(float)

#### subset columns

In [25]:
cable_theft_df.dtypes

sir_number                                                          object
year                                                                 int64
month                                                                int64
day                                                                  int64
hour                                                                 int64
reported_date                                                      float64
fy                                                                  object
week                                                                 int64
exchange                                                            object
exch_hits                                                            int64
css_db                                                              object
ned1                                                                object
damage_est_nu                                                       object
postcode                 

### create GCP json to load into bq

In [26]:
 def type_conversion(type_str):
    """ 
    Convert datatypes from pandas datatype to GCP datatype
    """
    #type_str = type_str.lower()
    if type_str== 'int64':
        return 'INT64'
    elif type_str== 'float64':
        return 'FLOAT64'
    elif type_str =='object':
        return 'STRING'
    elif type_str=='datetime64[ns]':
        return 'TIMESTAMP'
    elif type_str=='date':
        return 'DATE'
    else:
        print('unknown type')
        raise

In [27]:
dtype = []
for i in cable_theft_df.dtypes:
    dtype.append(type_conversion(i))

In [28]:
colname=cable_theft_df.columns
output = []

In [29]:
for col, dtype in  list(zip(colname, dtype)):
    output.append({
        "mode": "NULLABLE",
        "name": col,
        "type": dtype
    })

In [30]:
output

[{'mode': 'NULLABLE', 'name': 'sir_number', 'type': 'STRING'},
 {'mode': 'NULLABLE', 'name': 'year', 'type': 'INT64'},
 {'mode': 'NULLABLE', 'name': 'month', 'type': 'INT64'},
 {'mode': 'NULLABLE', 'name': 'day', 'type': 'INT64'},
 {'mode': 'NULLABLE', 'name': 'hour', 'type': 'INT64'},
 {'mode': 'NULLABLE', 'name': 'reported_date', 'type': 'FLOAT64'},
 {'mode': 'NULLABLE', 'name': 'fy', 'type': 'STRING'},
 {'mode': 'NULLABLE', 'name': 'week', 'type': 'INT64'},
 {'mode': 'NULLABLE', 'name': 'exchange', 'type': 'STRING'},
 {'mode': 'NULLABLE', 'name': 'exch_hits', 'type': 'INT64'},
 {'mode': 'NULLABLE', 'name': 'css_db', 'type': 'STRING'},
 {'mode': 'NULLABLE', 'name': 'ned1', 'type': 'STRING'},
 {'mode': 'NULLABLE', 'name': 'damage_est_nu', 'type': 'STRING'},
 {'mode': 'NULLABLE', 'name': 'postcode', 'type': 'STRING'},
 {'mode': 'NULLABLE', 'name': 'address', 'type': 'STRING'},
 {'mode': 'NULLABLE', 'name': 'information_about_incident', 'type': 'STRING'},
 {'mode': 'NULLABLE', 'name': '

In [31]:
schema_out_filename = output_path / "cable_theft_schema.json"

In [32]:
with open(schema_out_filename, 'w') as f2:
    json.dump(output, f2, indent=4)

In [33]:
out_filename = output_path / "cable_theft.parquet"

In [34]:
cable_theft_df.to_parquet(out_filename,index=False)

In [35]:
cable_theft_df = pd.read_parquet(out_filename)

In [36]:
cable_theft_df.head()

,sir_number,year,month,day,hour,reported_date,fy,week,exchange,exch_hits,css_db,ned1,damage_est_nu,postcode,address,information_about_incident,location_details,cut_theft,cable_size_total_pairs_only,live_pairs,total_length_metres,cable_affected_details,e_side_d_side_fibre,police_force,police_reference_number,nios_name_op_name,county,region,cable_size,indicative_copper_worth_based_on_current_uk_copper_price,cost_raptor,incident_form_rerturned_or_response_received_in_x_days,hrs_claimed,costs_excl_mhours,manhour_cost,total_cost,flagged_to_insurance,hrs_claimed_on_njr,number,est_number,pia,notes,date_3,date_mmm_yy,repeat_hit,cable_anchors_in_this_fin_year,incident_dt,day_of_week
0,11009105,2001,11,9,0,37204.0,2001-2002,32,Adlington,2,LC,NN-NORTHWESTEAST,None,PR6 9NA,"Bolton Road, Adlington, Chorley",Report states that cables were cut to facilita...,None,cut,20.0,20.0,0.0,None,d,Lancashire Constabulary,None,None,North West,Preston,< 100 pr,NaN,-,None,None,NaN,None,NaN,None,0.0,None,None,None,None,2001-09-11,2001 Wk: 32,1,0,2001-11-09,4
1,11005559,2002,8,16,0,37484.0,2002-2003,20,Horsham,9,SD,NL-WESTDOWNS,None,RH12 3RR,"Robin Hood Lane, Warnham, Horsham",Snare alarm activated and Police attended. Evi...,None,cut,1700.0,1700.0,0.0,None,e,Surrey Police,None,None,South East,Redhill,None,NaN,-,Not Sent,None,NaN,None,NaN,None,0.0,None,None,None,None,2002-08-16,2002 Wk: 20,1,0,2002-08-16,4
2,11005565,2002,8,16,0,37484.0,2002-2003,20,Downton,4,ST,NX-AVONWILTS,None,SP5 3HN,"Breamore Road, Downton, Salisbury",None,None,theft,600.0,600.0,0.0,None,e,Wiltshire Police,None,None,South West,Salisbury,None,NaN,-,Not Sent,None,NaN,None,NaN,None,0.0,None,None,None,None,2002-08-16,2002 Wk: 20,1,0,2002-08-16,4
3,7000132,2007,4,3,0,39175.0,2007-2008,1,Wrexham,4,WN,NW-CHESTERMERSEYSIDE,None,LL14 6PY,"Ruabon Road, Ruabon",Call to report two males seen in a white Renau...,None,intelligence,NaN,NaN,0.0,None,d,North Wales Police,None,None,Wales,Llandudno,None,NaN,-,None,None,NaN,None,NaN,None,0.0,None,None,None,None,2007-03-04,2007 Wk: 1,1,0,2007-04-03,1
4,7000158,2007,4,4,0,39176.0,2007-2008,1,Collyhurst,142,MR,NN-NORTHWESTWEST,None,M9 5RP,"Oakridge Walk,",Operation Eiger: The caller is reporting damag...,None,theft,NaN,NaN,0.0,None,e,Greater Manchester Police,None,None,North West,Manchester,None,NaN,-,Not Sent,None,NaN,None,NaN,None,0.0,None,None,None,None,2007-04-04,2007 Wk: 1,60,0,2007-04-04,2


In [37]:
# set project
# !gcloud config set project or-fttp-prope-aif-exp-prod
project_id = 'or-fttp-prope-aif-exp-prod'

In [38]:
# bigquery dataset
destination_dataset = "cable_theft"
destination_table_name = "cable_theft_history"

In [39]:
# Only need to run once to create dataset
#!bq --location=europe-west2 mk --dataset $destination_dataset

#### load data into bq from cloud storage

In [40]:
data_gcs_uri = "gs://or-fttp-prope-aif-exp-prod/cable_theft/data/output/cable_theft.parquet"

In [41]:
def load_data_to_bigquery(bq_client, json_schema_filepath, data_gcs_uri, destination_dataset, destination_table_name):
    with open(json_schema_filepath, 'r') as f:
        json_schema = json.load(f)
    job_config = bigquery.LoadJobConfig(
        schema=json_schema,
#        skip_leading_rows=0,
        write_disposition='WRITE_TRUNCATE',
        source_format=bigquery.SourceFormat.PARQUET,
    )
    
    destination  = f'{destination_dataset}.{destination_table_name}'
    
    load_job = bq_client.load_table_from_uri(
        data_gcs_uri, 
        destination=destination,
        job_config=job_config
    )  # Make an API request.

    load_job.result()  # Waits for the job to complete.

    destination_table = bq_client.get_table(destination) 
    print(f"Loaded {destination_table.num_rows} rows into {destination_table}")

In [42]:
load_bq_client = bigquery.Client(project=project_id)
load_data_to_bigquery(bq_client=load_bq_client,json_schema_filepath=schema_out_filename, data_gcs_uri=data_gcs_uri,destination_dataset=destination_dataset,destination_table_name=destination_table_name)

Loaded 11460 rows into or-fttp-prope-aif-exp-prod.cable_theft.cable_theft_history
